# Shadow Rollout with Seldon and Istio

This notebook shows how you can deploy "shadow" deployments to direct traffic not only to the main Seldon Deployment but also to a shadow deployment whose response will be dicarded. This allows you to test new models in a production setting and with production traffic and anlalyse how they perform before putting them live.

These are useful when you want to test a new model or higher latency inference piepline (e.g., with explanation components) with production traffic but without affecting the live deployment.


## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Istio Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Istio) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

In [ ]:
!kubectl create namespace seldon

namespace/seldon created


In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-kind" modified.


In [ ]:
from IPython.core.magic import register_line_cell_magic


@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, "w") as f:
        f.write(cell.format(**globals()))

Ensure the istio ingress gatewaty is port-forwarded to localhost:8004



* Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8004:8080`


In [ ]:
ISTIO_GATEWAY = "localhost:8004"

VERSION = !cat ../../../version.txt
VERSION = VERSION[0]
VERSION

'1.7.0-dev'

## Launch main model

We will create a very simple Seldon Deployment with a dummy model image `seldonio/mock_classifier:1.0`. This deployment is named `example`.

In [5]:
%%writetemplate model.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
  name: example
spec:
  name: production-model
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:{VERSION}
          imagePullPolicy: IfNotPresent
          name: classifier
        terminationGracePeriodSeconds: 1
    graph:
      children: []
      endpoint:
        type: REST
      name: classifier
      type: MODEL
    name: default
    replicas: 1


In [6]:
!kubectl apply -f model.yaml

seldondeployment.machinelearning.seldon.io/example configured


In [7]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=example -o jsonpath='{.items[0].metadata.name}')

deployment "example-default-0-classifier" successfully rolled out


### Get predictions

In [ ]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(
    deployment_name="example", namespace="seldon", gateway_endpoint=ISTIO_GATEWAY
)

#### REST Request

In [ ]:
r = sc.predict(gateway="istio", transport="rest")
assert r.success == True
print(r)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.6670563912281003
  }
}

Response:
{'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.09538308704053941]}}, 'meta': {'requestPath': {'classifier': 'seldonio/mock_classifier:1.7.0-dev'}}}


## Launch Shadow

We will now create a new Seldon Deployment for our Shadow deployment with a new model.

In [10]:
%%writetemplate shadow.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
  name: example
spec:
  name: shadow-model
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:{VERSION}
          imagePullPolicy: IfNotPresent
          name: classifier
        terminationGracePeriodSeconds: 1
    graph:
      children: []
      endpoint:
        type: REST
      name: classifier
      type: MODEL
    name: default
    replicas: 1
    traffic: 100
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:{VERSION}
          imagePullPolicy: IfNotPresent
          name: classifier
    graph:
      children: []
      endpoint:
        type: REST
      name: classifier
      type: MODEL
    name: shadow
    replicas: 1
    shadow: true
    traffic: 100



You can use the traffic field under the shadow componentSpecs to mirror a fraction of the traffic, instead of mirroring all requests. If this field is absent, all traffic will be mirrored. 

In [11]:
!kubectl apply -f shadow.yaml

seldondeployment.machinelearning.seldon.io/example configured


In [12]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=example -o jsonpath='{.items[0].metadata.name}')
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=example -o jsonpath='{.items[1].metadata.name}')

deployment "example-default-0-classifier" successfully rolled out
Waiting for deployment "example-shadow-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "example-shadow-0-classifier" successfully rolled out


Let's send a bunch of requests to the endpoint.

In [13]:
for i in range(10):
    r = sc.predict(gateway="istio", transport="rest")

In [14]:
default_count = !kubectl logs $(kubectl get pod -lseldon-app=example-default -o jsonpath='{.items[0].metadata.name}') classifier | grep "root.predict" | wc -l

In [15]:
shadow_count = !kubectl logs $(kubectl get pod -lseldon-app=example-shadow -o jsonpath='{.items[0].metadata.name}') classifier | grep "root.predict" | wc -l

In [16]:
print(shadow_count)
print(default_count)
assert int(shadow_count[0]) == 10
assert int(default_count[0]) == 11

['10']
['11']


## TearDown

In [17]:
!kubectl delete -f model.yaml

seldondeployment.machinelearning.seldon.io "example" deleted
